<a href="https://colab.research.google.com/github/Rizwankaka/Agentic-AI-/blob/main/Pydantic/pydanticai_research_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a Research Agent with Pydantic AI

In [2]:
%pip install -qU pydantic-ai nest_asyncio devtools duckduckgo-search tavily-python

In [3]:
import os
from google.colab import userdata
from IPython.display import display, Markdown

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY') # If you dont have this use the DDGS alternative below

In [5]:
import nest_asyncio
nest_asyncio.apply()

## Setting Up search Tavily & DDGS

I've put both Tavily search and DuckDuckGo search here. You should use DuckDuckGo search if you want a free alternative, but you can get an API key with Tavily which will also give you a number of free calls. Unfortunately, nowadays with DuckDuckGo search you often hit rate limits if you're not using a proxy system.

No need to use both, just using one of these is fine.

### DuckDuckGo Search

In [7]:
from duckduckgo_search import DDGS

results = DDGS().text("python programming", max_results=5)
print(results)

[{'title': 'Welcome to Python.org', 'href': 'https://www.python.org/', 'body': "Python is a programming language that lets you work quickly and integrate systems more effectively. Learn More. Get Started. Whether you're new to programming or an experienced developer, it's easy to learn and use Python. Start with our Beginner's Guide. Download."}, {'title': 'Python For Beginners | Python.org', 'href': 'https://www.python.org/about/gettingstarted/', 'body': 'Learn how to get started with Python, a popular and easy-to-use programming language. Find out how to install, edit, and use Python, and explore its libraries, documentation, and community resources.'}, {'title': 'Python Tutorial - W3Schools', 'href': 'https://www.w3schools.com/python/', 'body': 'W3Schools offers a comprehensive and interactive Python tutorial with examples, exercises, quizzes, and references. You can learn Python basics, web applications, file handling, database handling, and more.'}, {'title': 'Python (programming 

In [9]:
results = DDGS().text("describe the characteristic habits and behaviors of humans as a species")
print(results)

[{'title': '1.4 Human Origins and Adaptations Flashcards - Quizlet', 'href': 'https://quizlet.com/90665484/14-human-origins-and-adaptations-flash-cards/', 'body': 'Study with Quizlet and memorize flashcards containing terms like Q1: Explain why evolution is relevant to understanding human form and function., Q2: Describe evolution and natural selection., Q3: Describe some human characteristics that can be attributed to the tree-dwelling habits or earlier primates. and more.'}, {'title': 'Human behavior | Definition, Theories, Characteristics, Examples, Types ...', 'href': 'https://www.britannica.com/topic/human-behavior', 'body': 'Human behavior, the potential and expressed capacity for physical, mental, and social activity throughout human life. Humans, like other animal species, have a typical life course that consists of successive phases of growth, each characterized by a distinct set of physical, physiological, and behavioral features.'}, {'title': '22 Characteristics of Human Bei

### Tavily Search

In [10]:
from tavily import TavilyClient, AsyncTavilyClient

# Setup the Tavily Client
tavily_client = AsyncTavilyClient(api_key=os.environ["TAVILY_API_KEY"])

# Simple Search
response = await tavily_client.search("Who is Leo Messi?", max_results=3)

print(response['results'])

[{'title': 'Career of Lionel Messi - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Career_of_Lionel_Messi', 'content': "Lionel Messi is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team.His individual achievements include eight Ballon d'Or awards, the most for any footballer. Having won 45 team trophies, [note 1] he is the most decorated player in the history of professional football. [11]", 'score': 0.84027237, 'raw_content': None}, {'title': 'Lionel Messi - Wikipedia', 'url': 'https://en.wikipedia.org/wiki/Lionel_Messi', 'content': 'He scored twice in the last group match, a 3–2 victory over Nigeria, his second goal coming from a free kick, as they finished first in their group.[423] Messi assisted a late goal in extra time to ensure a 1–0 win against Switzerland in the round of 16, and played in the 1–0 quarter-final win against Belgium as Argentina progressed to the semi-final o

In [11]:
# Responses
for result in response['results']:
    print(result['title'])
    print(result['content'])
    print('\n')

Career of Lionel Messi - Wikipedia
Lionel Messi is an Argentine professional footballer who plays as a forward for and captains both Major League Soccer club Inter Miami and the Argentina national team.His individual achievements include eight Ballon d'Or awards, the most for any footballer. Having won 45 team trophies, [note 1] he is the most decorated player in the history of professional football. [11]


Lionel Messi - Wikipedia
He scored twice in the last group match, a 3–2 victory over Nigeria, his second goal coming from a free kick, as they finished first in their group.[423] Messi assisted a late goal in extra time to ensure a 1–0 win against Switzerland in the round of 16, and played in the 1–0 quarter-final win against Belgium as Argentina progressed to the semi-final of the World Cup for the first time since 1990.[424][425] Following a 0–0 draw in extra time, they eliminated the Netherlands 4–2 in a penalty shootout to reach the final, with Messi scoring his team's first pen

In [12]:
# RAG Context search

# Context Search
context = await tavily_client.get_search_context(query="Who is Leo Messi?", max_results=3)

print(context)

"[\"{\\\"url\\\": \\\"https://en.wikipedia.org/wiki/Lionel_Messi\\\", \\\"content\\\": \\\"He scored twice in the last group match, a 3\\\\u20132 victory over Nigeria, his second goal coming from a free kick, as they finished first in their group.[423] Messi assisted a late goal in extra time to ensure a 1\\\\u20130 win against Switzerland in the round of 16, and played in the 1\\\\u20130 quarter-final win against Belgium as Argentina progressed to the semi-final of the World Cup for the first time since 1990.[424][425] Following a 0\\\\u20130 draw in extra time, they eliminated the Netherlands 4\\\\u20132 in a penalty shootout to reach the final, with Messi scoring his team's first penalty.[426]\\\\nBilled as Messi versus Germany, the world's best player against the best team, the final was a repeat of the 1990 final featuring Diego Maradona.[427] Within the first half-hour, Messi had started the play that led to a goal, but it was ruled offside. \\\\\\\"[582] Moreover, several pundit

## Setting up the Agent & Tools

In [13]:
from __future__ import annotations as _annotations

import asyncio
import os
from dataclasses import dataclass
from typing import Any

from devtools import debug
from httpx import AsyncClient
import datetime

from pydantic_ai import Agent, ModelRetry, RunContext
from pydantic import BaseModel, Field

In [14]:
@dataclass
class SearchDataclass:
    max_results: int
    todays_date: str

@dataclass
class ResearchDependencies:
    todays_date: str

class ResearchResult(BaseModel):
    research_title: str = Field(description='This is a top level Markdown heading that covers the topic of the query and answer prefix it with #')
    research_main: str = Field(description='This is a main section that provides answers for the query and research')
    research_bullets: str = Field(description='This is a set of bulletpoints that summarize the answers for query')

In [15]:
## Make the agent
search_agent = Agent('openai:gpt-4o',
                     deps_type=ResearchDependencies,
                     result_type=ResearchResult,
                     system_prompt='Your a helpful research assistant, you are an expert in research '
                     'If you are given a question you write strong keywords to do 3-5 searches in total '
                     '(each with a query_number) and then combine the results' )


In [19]:
# Check if the tool already exists, and only register if it doesn't
if 'get_search' not in search_agent._function_tools:
    @search_agent.tool #Tavily
    async def get_search(search_data:RunContext[SearchDataclass],query: str, query_number: int) -> dict[str, Any]:
        """Get the search for a keyword query.

        Args:
            query: keywords to search.
        """
        print(f"Search query {query_number}: {query}")
        max_results = search_data.deps.max_results
        results = await tavily_client.get_search_context(query=query, max_results=max_results)

        return results
else:
    print("Tool 'get_search' already registered.")

Tool 'get_search' already registered.


In [20]:
## set up the dependencies

# Get the current date
current_date = datetime.date.today()

# Convert the date to a string
date_string = current_date.strftime("%Y-%m-%d")


deps = SearchDataclass(max_results=3, todays_date=date_string)

In [21]:
result = await search_agent.run(
    'can you give me a very detailed bio of Sam Altman?', deps=deps
)

Search query 1: Sam Altman biography
Search query 2: Sam Altman early life and education
Search query 3: Sam Altman career
Search query 4: Sam Altman investments and achievements
Search query 5: Sam Altman personal life


In [22]:
print(result.data)

research_title='Detailed Biography of Sam Altman' research_main='Sam Altman was born on April 22, 1985, in Chicago, Illinois, and raised in St. Louis, Missouri. From an early age, he showed interest in technology and entrepreneurship, setting the stage for his future endeavors. Altman attended Stanford University to study computer science but did not complete his degree, opting instead to pursue business ventures in the tech field.\n\nIn 2005, he co-created Loopt, a location-based social networking app, which was acquired by Green Dot Corporation for $43.4 million. This marked the beginning of Altman\'s ascent in Silicon Valley. His true influence became apparent during his tenure as the president of Y Combinator, one of the most prominent startup incubators globally. He was instrumental in scaling the organization and expanding the types of companies it supported, ultimately nurturing industry giants like Airbnb and Dropbox.\n\nIn 2019, Altman transitioned to a full-time role at OpenA

In [23]:
result.data.research_title = '#'+result.data.research_title
print(result.data.research_title)

#Detailed Biography of Sam Altman


In [24]:
print(result.data.research_main)

Sam Altman was born on April 22, 1985, in Chicago, Illinois, and raised in St. Louis, Missouri. From an early age, he showed interest in technology and entrepreneurship, setting the stage for his future endeavors. Altman attended Stanford University to study computer science but did not complete his degree, opting instead to pursue business ventures in the tech field.

In 2005, he co-created Loopt, a location-based social networking app, which was acquired by Green Dot Corporation for $43.4 million. This marked the beginning of Altman's ascent in Silicon Valley. His true influence became apparent during his tenure as the president of Y Combinator, one of the most prominent startup incubators globally. He was instrumental in scaling the organization and expanding the types of companies it supported, ultimately nurturing industry giants like Airbnb and Dropbox.

In 2019, Altman transitioned to a full-time role at OpenAI, focusing on artificial intelligence development. Under his leadersh

In [25]:
print(result.data.research_bullets)

- Sam Altman was born on April 22, 1985, in Chicago, Illinois, and raised in St. Louis, Missouri.
- Attended Stanford University but dropped out to focus on technology ventures.
- Co-founded Loopt in 2005; it was acquired for $43.4 million.
- Became president of Y Combinator, nurturing startups like Airbnb and Dropbox.
- Transitioned full-time to OpenAI in 2019, advancing AI technology with projects like ChatGPT.
- Notable investor in Airbnb, Stripe, Reddit, emphasizing innovation and long-term growth.
- Named one of "Time" magazine's 100 most influential people in 2023.
- Balances professional achievements with personal life through mindfulness and family values.


In [26]:
combined_markdown = "\n\n".join([result.data.research_title, result.data.research_main, result.data.research_bullets])

Markdown(combined_markdown)

#Detailed Biography of Sam Altman

Sam Altman was born on April 22, 1985, in Chicago, Illinois, and raised in St. Louis, Missouri. From an early age, he showed interest in technology and entrepreneurship, setting the stage for his future endeavors. Altman attended Stanford University to study computer science but did not complete his degree, opting instead to pursue business ventures in the tech field.

In 2005, he co-created Loopt, a location-based social networking app, which was acquired by Green Dot Corporation for $43.4 million. This marked the beginning of Altman's ascent in Silicon Valley. His true influence became apparent during his tenure as the president of Y Combinator, one of the most prominent startup incubators globally. He was instrumental in scaling the organization and expanding the types of companies it supported, ultimately nurturing industry giants like Airbnb and Dropbox.

In 2019, Altman transitioned to a full-time role at OpenAI, focusing on artificial intelligence development. Under his leadership, OpenAI introduced groundbreaking AI technologies, including ChatGPT, pushing the boundaries of AI research and applications. Altman is known for his investment acumen, with notable investments in early-stage companies such as Airbnb, Stripe, and Reddit. His investment philosophy emphasizes innovation, the quality of founding teams, and a long-term view.

Throughout his career, Altman has received various accolades, including being named one of "Time" magazine's 100 most influential people in 2023. His commitment to technology's ethical development is evident in his support for projects like YC Research, which explores future social paradigms like universal basic income. Beyond technology, Altman has shown interest in biotech and nuclear energy, further diversifying his influence and investment portfolio.

On a personal note, Altman maintains a balance between his professional and personal life, emphasizing mindfulness and family values as pivotal aspects of his life. His journey continues to inspire many in the startup ecosystem worldwide.

- Sam Altman was born on April 22, 1985, in Chicago, Illinois, and raised in St. Louis, Missouri.
- Attended Stanford University but dropped out to focus on technology ventures.
- Co-founded Loopt in 2005; it was acquired for $43.4 million.
- Became president of Y Combinator, nurturing startups like Airbnb and Dropbox.
- Transitioned full-time to OpenAI in 2019, advancing AI technology with projects like ChatGPT.
- Notable investor in Airbnb, Stripe, Reddit, emphasizing innovation and long-term growth.
- Named one of "Time" magazine's 100 most influential people in 2023.
- Balances professional achievements with personal life through mindfulness and family values.

In [27]:
debug(result)

<ipython-input-27-4217a35008bd>:1 <cell line: 0>
    result: RunResult(
        _all_messages=[
            ModelRequest(
                parts=[
                    SystemPromptPart(
                        content=(
                            'Your a helpful research assistant, you are an expert in research If you are given a quest'
                            'ion you write strong keywords to do 3-5 searches in total (each with a query_number) and '
                            'then combine the results'
                        ),
                        dynamic_ref=None,
                        part_kind='system-prompt',
                    ),
                    UserPromptPart(
                        content='can you give me a very detailed bio of Sam Altman?',
                        timestamp=datetime.datetime(2025, 1, 19, 16, 57, 13, 993103, tzinfo=datetime.timezone.utc),
                        part_kind='user-prompt',
                    ),
                ],
               

RunResult(_all_messages=[ModelRequest(parts=[SystemPromptPart(content='Your a helpful research assistant, you are an expert in research If you are given a question you write strong keywords to do 3-5 searches in total (each with a query_number) and then combine the results', dynamic_ref=None, part_kind='system-prompt'), UserPromptPart(content='can you give me a very detailed bio of Sam Altman?', timestamp=datetime.datetime(2025, 1, 19, 16, 57, 13, 993103, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[ToolCallPart(tool_name='get_search', args=ArgsJson(args_json='{"query": "Sam Altman biography", "query_number": 1}'), tool_call_id='call_MT08YIwumWaj2In0pFHkwieE', part_kind='tool-call'), ToolCallPart(tool_name='get_search', args=ArgsJson(args_json='{"query": "Sam Altman early life and education", "query_number": 2}'), tool_call_id='call_Gj6qvBWjjuu3BGA2CKXvv4YD', part_kind='tool-call'), ToolCallPart(tool_name='get_search', args=ArgsJson(ar

In [28]:
result = await search_agent.run(
    'What is the latest AI News?', deps=deps
)

Search query 1: latest AI news October 2023
Search query 2: recent AI technology updates 2023
Search query 3: AI advancements October 2023


In [29]:
print(result.data)

research_title='Latest Developments in Artificial Intelligence - October 2023' research_main="October 2023 has witnessed substantial advancements in the field of Artificial Intelligence (AI) across various areas. Key developments include the launch of significant AI models and products, major investments, and regulatory efforts.\n\nOne of the highlights for October was OpenAI's rollout of DALL-E 3 and GPTs along with ChatGPT-4 Turbo. These releases continue to expand the capabilities of generative AI technologies. In a parallel move, Google has agreed to invest up to $2 billion in Anthropic, a significant financial commitment aimed at collaboration and advancement in AI safety.\n\nMoreover, the regulatory landscape saw a notable change with President Biden's signing of an Executive Order focused on the safe and secure development and usage of AI, underlining the government's proactive stance on responsible AI deployment.\n\nTechnological advancements also included Google's DeepMind ach

## Add in the date

In [30]:
@search_agent.system_prompt
async def add_current_date(ctx: RunContext[ResearchDependencies]) -> str:
    todays_date = ctx.deps.todays_date
    system_prompt=f'Your a helpful research assistant, you are an expert in research \
                If you are given a question you write strong keywords to do 3-5 searches in total \
                (each with a query_number) and then combine the results \
                if you need todays date it is {todays_date}'
    return system_prompt

In [31]:
result = await search_agent.run(
    'What are the major AI News announcements in the last few days?', deps=deps
)

Search query 1: Major AI news announcements January 2025
Search query 2: AI breakthroughs January 2025
Search query 3: Recent AI technology updates 2025


In [32]:
print(result.data)

research_title='Major AI News Announcements - January 2025' research_main="In January 2025, several exciting developments have occurred in the field of artificial intelligence, reflecting both technological advancements and strategic industry shifts.\n- At CES 2025, multiple major companies showcased groundbreaking AI technologies. Intel announced its new AI-powered PC architecture employing their latest Lunar Lake processors, which integrate Neural Processing Units (NPUs) to enhance device intelligence. NVIDIA's CEO, Jensen Huang, introduced the Cosmos world foundation model platform that aims to revolutionize robotics and industrial AI, coupled with significant advancements for autonomous vehicles through the use of their Omniverse and Cosmos technologies.\n- Google's DeepMind has introduced new methods to understand AI 'minds', potentially offering insights into how AI models function and improve transparency and trustworthiness in AI applications.\n- Major AI trends for 2025 have a

In [33]:
result.data.research_title = '#'+result.data.research_title
print(result.data.research_title)

#Major AI News Announcements - January 2025


In [34]:
print(result.data.research_main)

In January 2025, several exciting developments have occurred in the field of artificial intelligence, reflecting both technological advancements and strategic industry shifts.
- At CES 2025, multiple major companies showcased groundbreaking AI technologies. Intel announced its new AI-powered PC architecture employing their latest Lunar Lake processors, which integrate Neural Processing Units (NPUs) to enhance device intelligence. NVIDIA's CEO, Jensen Huang, introduced the Cosmos world foundation model platform that aims to revolutionize robotics and industrial AI, coupled with significant advancements for autonomous vehicles through the use of their Omniverse and Cosmos technologies.
- Google's DeepMind has introduced new methods to understand AI 'minds', potentially offering insights into how AI models function and improve transparency and trustworthiness in AI applications.
- Major AI trends for 2025 have also been identified, highlighting responsible AI and regulatory approaches, su

In [35]:
print(result.data.research_bullets)

- Intel's Lunar Lake processors with integrated NPUs announced at CES 2025.
- NVIDIA introduced the Cosmos world foundation model platform for robotics.
- Google DeepMind working on understanding AI 'minds'.
- China's national AI standards and decentralized U.S. regulatory approaches.
- Rise of open-source AI models, e.g., Deepseek.
- Increasing AI-driven workspace productivity.
- Growth of collaborative AI and multimodal models in education and research.


In [36]:
combined_markdown = "\n\n".join([result.data.research_title, result.data.research_main, result.data.research_bullets])

Markdown(combined_markdown)

#Major AI News Announcements - January 2025

In January 2025, several exciting developments have occurred in the field of artificial intelligence, reflecting both technological advancements and strategic industry shifts.
- At CES 2025, multiple major companies showcased groundbreaking AI technologies. Intel announced its new AI-powered PC architecture employing their latest Lunar Lake processors, which integrate Neural Processing Units (NPUs) to enhance device intelligence. NVIDIA's CEO, Jensen Huang, introduced the Cosmos world foundation model platform that aims to revolutionize robotics and industrial AI, coupled with significant advancements for autonomous vehicles through the use of their Omniverse and Cosmos technologies.
- Google's DeepMind has introduced new methods to understand AI 'minds', potentially offering insights into how AI models function and improve transparency and trustworthiness in AI applications.
- Major AI trends for 2025 have also been identified, highlighting responsible AI and regulatory approaches, such as China's establishment of over 50 national and industry standards focused on machine learning by 2026. The United States emphasizes a decentralized regulatory framework.
- Open-source AI has gained momentum with models like Deepseek, providing affordable and high-performance AI tools that could reshape global AI policies and address cybersecurity, privacy, and ethical challenges.
- AI's impact on the workspace continues to grow with innovations that promise to increase productivity by automating routine tasks.
- Collaborative AI systems and multimodal AI models are also becoming center-stage, with AI agents working together to tackle complex tasks alongside human interactions.

- Intel's Lunar Lake processors with integrated NPUs announced at CES 2025.
- NVIDIA introduced the Cosmos world foundation model platform for robotics.
- Google DeepMind working on understanding AI 'minds'.
- China's national AI standards and decentralized U.S. regulatory approaches.
- Rise of open-source AI models, e.g., Deepseek.
- Increasing AI-driven workspace productivity.
- Growth of collaborative AI and multimodal models in education and research.